In [5]:
from PIL import Image
from PIL import ImageDraw
import cv2
Image.MAX_IMAGE_PIXELS = None

In [ ]:
width_mm = 420  # A3 paper width in mm
height_mm = 297  # A3 paper height in mm
dpi = 1200  # scanning resolution in dots per inch

# Convert mm to inches
width_in = width_mm / 25.4
height_in = height_mm / 25.4

# Calculate the number of pixels
a3_w = int(width_in * dpi)
a3_h = int(height_in * dpi)

print(a3_w, a3_h)  

In [ ]:
# Calculate the dimensions of letter paper in mm
letter_width_mm = 215.9  # Letter paper width in mm (8.5 inches)
letter_height_mm = 279.4  # Letter paper height in mm (11 inches)

# Convert letter dimensions to inches
letter_width_in = letter_width_mm / 25.4
letter_height_in = letter_height_mm / 25.4

# Calculate the number of pixels for letter dimensions
l_w = int(letter_width_in * dpi)
l_h = int(letter_height_in * dpi)

print(l_w, l_h)

In [8]:
def create_letter_grid(dpi=1200, grid_size_mm=10, margin_mm=None):
    # Letterサイズ（単位：mm）
    letter_width_mm, letter_height_mm = 215.5, 279.4

    # mmをピクセルに変換
    pixels_per_mm = dpi / 25.4
    width_px = int(letter_width_mm * pixels_per_mm)
    height_px = int(letter_height_mm * pixels_per_mm)
    grid_size_px = int(grid_size_mm * pixels_per_mm)

    margin_px = 0
    if margin_mm is not None:
        margin_px = int(margin_mm * pixels_per_mm)

    # 白色の背景で新しい画像を作成
    image = Image.new('RGB', (width_px, height_px), color='white')
    draw = ImageDraw.Draw(image)

    # マージンを考慮して格子線を描画
    for x in range(margin_px, width_px - margin_px, grid_size_px):
        draw.line([(x, margin_px), (x, height_px - margin_px)], fill=(50,50,50), width=5)
    for y in range(margin_px, height_px - margin_px, grid_size_px):
        draw.line([(margin_px, y), (width_px - margin_px, y)], fill=(50,50,50), width=5)

    # # マージン領域を示す線を描画（オプション）
    # draw.rectangle([margin_px, margin_px, width_px - margin_px, height_px - margin_px], outline='gray', width=2)

    return image

# grid_image = create_letter_grid(dpi=1200, grid_size_mm=10, margin_mm=5)
grid_image = create_letter_grid(dpi=1200, grid_size_mm=10)

grid_image_rot = grid_image.rotate(0.1, expand=True)
# grid_image_rot.show()

In [9]:
# A3黒画像
a3_image = Image.new('RGB', (a3_w,a3_h), 'black')
a3_image.paste(grid_image_rot,(200, 200))

# a3_image.show()


# Save or display the image as needed
# image.show()  # Uncomment to display the image
a3_image.save('sample.tif',format="tiff",dpi=(1200,1200))  # Uncomment to save the image

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 画像をグレースケールで読み込む
image = cv2.imread('sample.tif', cv2.IMREAD_GRAYSCALE)
print(image.shape)

# 画像を行方向に10等分する
row_parts = 5
col_parts = 8
height, width = image.shape
part_height = height // row_parts
part_width = width // col_parts

l_coords = []
r_coords = []
t_coords = []
b_coords = []

# 左右の境界線を検出する
for i in range(row_parts):
    # 現在のパートの開始行と終了行のインデックスを計算する
    start_row = i * part_height

    # 現在のパートの平均輝度値を計算する
    brightness = image[start_row, :]

    # 輝度値の導関数を計算する
    derivative = np.gradient(brightness)

    # 導関数が最初から数えて50を超えた要素の要素番号を取得
    indices = np.where(derivative > 50)[0]
    # for x in indices:
    if indices.size > 0:
        l_coords.append((int(indices[0]), start_row))

    # 導関数が後ろから数えて-50を下回った要素の要素番号を取得
    reverse_indices = np.where(derivative[::-1] < -50)[0]
    indices = len(derivative) - reverse_indices - 1
    if indices.size > 0:
        r_coords.append((int(indices[0]), start_row))

# 上下の境界線を検出する
for i in range(col_parts):
    # 現在のパートの開始列と終了列のインデックスを計算する
    start_col = i * part_width

    # 現在のパートの平均輝度値を計算する
    brightness = image[:,start_col]

    # 輝度値の導関数を計算する
    derivative = np.gradient(brightness)

    # 導関数が最初から数えて50を超えた要素の要素番号を取得
    indices = np.where(derivative > 50)[0]
    # for x in indices:
    if indices.size > 0:
        t_coords.append((start_col, int(indices[0])))

    # 導関数が後ろから数えて-50を下回った要素の要素番号を取得
    reverse_indices = np.where(derivative[::-1] < -50)[0]
    indices = len(derivative) - reverse_indices - 1
    if indices.size > 0:
        b_coords.append((start_col, int(indices[0])))

print(l_coords)
print(r_coords)
print(t_coords)
print(b_coords)
# Fit an approximation line to the l_coords data
x_l_coords, y_l_coords = zip(*l_coords)
l_line = np.polyfit(x_l_coords, y_l_coords, 1)

x_r_coords, y_r_coords = zip(*r_coords)
r_line = np.polyfit(x_r_coords, y_r_coords, 1)

x_t_coords, y_t_coords = zip(*t_coords)
t_line = np.polyfit(x_t_coords, y_t_coords, 1)

x_b_coords, y_b_coords = zip(*b_coords)
b_line = np.polyfit(x_b_coords, y_b_coords, 1)

origin_image = cv2.imread('sample.tif')
s_l_x = 0
s_l_y = int(l_line[0]*s_l_x + l_line[1])
e_l_x = width
e_l_y = int(l_line[0]*e_l_x + l_line[1])
s_r_x = 0
s_r_y = int(r_line[0]*s_r_x + r_line[1])
e_r_x = width
e_r_y = int(r_line[0]*e_r_x + r_line[1])

s_t_x = 0
s_t_y = int(t_line[0]*s_t_x + t_line[1])
e_t_x = width
e_t_y = int(t_line[0]*e_t_x + t_line[1])
s_b_x = 0
s_b_y = int(b_line[0]*s_b_x + b_line[1])
e_b_x = width
e_b_y = int(b_line[0]*e_b_x + b_line[1])

cv2.line(origin_image, (s_l_x, s_l_y), (e_l_x, e_l_y), (0,0,255), 20)
cv2.line(origin_image, (s_r_x, s_r_y), (e_r_x, e_r_y), (0,0,255), 20)
cv2.line(origin_image, (s_t_x, s_t_y), (e_t_x, e_t_y), (0,0,255), 20)
cv2.line(origin_image, (s_b_x, s_b_y), (e_b_x, e_b_y), (0,0,255), 20)

for coord in l_coords:
    cv2.circle(origin_image, coord, 50, (0, 255, 0), -1)

for coord in r_coords:
    cv2.circle(origin_image, coord, 50, (0, 255, 0), -1)

for coord in t_coords:
    cv2.circle(origin_image, coord, 50, (0, 255, 0), -1)

for coord in b_coords:
    cv2.circle(origin_image, coord, 50, (0, 255, 0), -1)

# cv2.imwrite('output_addLine.tif', origin_image)


# addline_image = Image.open('output_addLine.tif')
# addline_image.show()  


angle = np.rad2deg(np.arctan(t_line[0]))
print(f"angle: {angle}")

In [ ]:
def line_intersection(line1, line2):
    """
    2つの直線の交点を計算する関数
    line1, line2: [a, b] の形式で、y = ax + b の直線を表す
    """
    a1, b1 = line1
    a2, b2 = line2
    
    if a1 == a2:
        return None  # 平行線の場合
    
    x = (b2 - b1) / (a1 - a2)
    y = a1 * x + b1
    return int(x), int(y)


# 交点の計算
tl_coords = line_intersection(t_line, l_line)
bl_coords = line_intersection(b_line, l_line)
br_coords = line_intersection(b_line, r_line)
tr_coords = line_intersection(t_line, r_line)

cv2.circle(origin_image, tl_coords, 50, (255, 0, 0), -1)
cv2.circle(origin_image, bl_coords, 50, (255, 0, 0), -1)
cv2.circle(origin_image, br_coords, 50, (255, 0, 0), -1)
cv2.circle(origin_image, tr_coords, 50, (255, 0, 0), -1)

# cv2.imwrite('output_intersection.tif', origin_image)

# intersection_image = Image.open('output_intersection.tif')
# intersection_image.show()  


In [12]:
x = min(tl_coords[0], bl_coords[0]) 
y = min(tr_coords[1], tl_coords[1])
w = max(br_coords[0] , tr_coords[0]) - x
h = max(bl_coords[1], br_coords[1]) - y

# draw rectangle on the image
cv2.rectangle(origin_image, (x, y), (x+w, y+h), (0, 255, 0), 20)

cv2.imwrite('output_rect.tif', origin_image)

output_image = Image.open('output_rect.tif')
output_image.show()

In [ ]:
# crop the image
image = cv2.imread('sample.tif')
cropped_image = image[y:y+h, x:x+w]
cv2.imwrite('output_cropped.tif', cropped_image)
